In [1]:
import numpy as np
from random import uniform, shuffle
import csv

def random():
    return round(np.random.random(),2)

def pred_runs(a,b,random):
    k='^'+make_reg(a)+', '+make_reg(b)
    cum_prob = ipl.filter(regex=k,axis=0)
    if len(cum_prob)>1:
        s = cum_prob.filter(regex='.*' + b + '$',axis=0)
        t= cum_prob.filter(regex='^'+ a +'.*',axis=0)
        if len(s)==1:
            cum_prob=s
            b = '.*' + b + '$'
        elif len(t) ==1:
            cum_prob=t
            a = '^'+ a +'.*'
#     return cum_prob
#     print()
#     print()
#     print(a,b)
    try:
        if ((cum_prob.iloc[:,7]<5).bool()):
            #print('lesser than 5')
            raise
        else:
            for i in range(0,6):
                if((random<=cum_prob.iloc[:,i]).bool()):
                    if(i==5):
                        return 6
                    else:
                        return i
    except:
        try:
            batclus = int(batsman_clus.filter(regex=a,axis=0)['prediction'])
#             print(batsman_clus.filter(regex=a,axis=0)['prediction'])
            bowlclus = int(bowler_clus.filter(regex=b,axis=0)['prediction'])
        except:
            batclus = 1
            bowlclus = 1
        cum_prob = clusters_pred.filter(like=str(batclus)+', '+str(bowlclus), axis =0)
        #print(batclus,bowlclus,cum_prob)
        for i in range(0,6):
                    if((random<=cum_prob.iloc[:,i]).bool()):
                        if(i==5):
                            return 6
                        else:
                            return i
    return 6
                    
def pred_wicket(a,b,cur):
    k='^'+make_reg(a)+', '+make_reg(b)
    try:
        if int(ipl.filter(regex=k,axis=0)['balls'])<5:
            raise
        stay_rate = cur*(1-float(ipl.filter(regex=k,axis=0)['outs']))
        return stay_rate
    except:
        #print(a)
        #print(b)
        try:
            batclus = int(batsman_clus.filter(like=a,axis=0)['prediction'])
            bowlclus = int(bowler_clus.filter(like=b,axis=0)['prediction'])
        except:
            batclus = 1
            bowlclus = 1
        stay_rate=cur*(1-float(clusters_pred.filter(like=str(batclus)+', '+str(bowlclus), axis =0)['outs']))
        return stay_rate

def innings(batsman, bowlers, file_name):
    d_batsmen = dict()
    d_bowler = dict()
    with open(file_name+'.csv','w') as csvfile:
        fieldnames = ['batsman','bowler','runs','out']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        writer.writeheader()
        strike = dict()
        non_strike = dict()

        # Team 1 batting
        strike['name'] = batsman[0]
        strike['stay_rate'] = 1
        non_strike['name'] = batsman[1]
        non_strike['stay_rate'] = 1


        wicket_count = 0
        wicket = 0
        flag_allout = True
        total = 0
        over_number = 1
        while over_number <= 20 and flag_allout:
            over_run = 0
            over_wicket = 0
            print("Over Number ",over_number)
            print("The Bowler is", bowlers[over_number%len(bowlers)])
            bowler = bowlers[over_number%len(bowlers)]
            for balls in range(1,7):
                wicket = 0
                runs = pred_runs(strike['name'], bowlers[over_number%len(bowlers)],random())
                strike['stay_rate'] = pred_wicket(strike['name'], bowlers[over_number%len(bowlers)],strike['stay_rate'])
                
                
                if strike['stay_rate'] < uniform(0.3,0.55):
                    wicket_count = wicket_count + 1
                    wicket = 1
                    over_wicket +=1
                    writer.writerow({'batsman':strike['name'],'bowler':bowler,'runs':runs,'out':wicket})
                    d_bowler[bowler] = d_bowler.get(bowler,0)+1
                    print(strike['name']+" got out")
                    if wicket_count == 10:
                        print("Innings Complete!")
                        flag_allout = False
                        break

                    strike['name'] = batsman[wicket_count + 1]
                    print(strike['name'] +" is the man next in")
                    strike['stay_rate'] = 1
                    #print(total,"/",wicket_count)
                if wicket == 0:    
                    writer.writerow({'batsman':strike['name'],'bowler':bowler,'runs':runs,'out':wicket})
                    d_batsmen[strike['name']] = d_batsmen.get(strike['name'],0)+runs
                    over_run+=runs
                    print(strike['name']+" Has scored", runs)
                if runs == 1 or runs == 3:
                        #print(total,"/",wicket_count)
                        strike, non_strike = non_strike, strike
                total+=runs
                
            over_number = over_number  + 1
            strike, non_strike = non_strike, strike
            print()
            print("OVER STATS")
            print("runs:",over_run,", outs",over_wicket)
            print()
        print()
        print('BATSMAN STATS')
        print(d_batsmen)
        print('BOWLER STATS')
        print(d_bowler)
        print()
        print('INNINGS OVER')
        return total
    
    
from random import shuffle
def complete_bowling_order(bowlers):
    over_count = dict()
    for bowler in bowlers:
        over_count[bowler] = over_count.get(bowler,0)+1
    fill_bowler = list()
    for key,value in over_count.items():
        if value < 4:
            fill_bowler.extend([key]*(4-value))
    shuffle(fill_bowler)
    bowlers.extend(fill_bowler[:20-len(bowlers)])
    return bowlers


In [2]:
def make_reg(temp):
    temp=temp.split()
    reg=temp[0][0]+'.*'+temp[-1]
    return reg

In [3]:
import pandas as pd

In [4]:
clusters_pred = pd.read_csv('final_cluster.csv') 
batsman_clus = pd.read_csv('batsman_cluster.csv')
bowler_clus =pd.read_csv('bowler_cluster.csv')
ipl = pd.read_csv('new.csv')
ipl = ipl.set_index('key')
clusters_pred = clusters_pred.set_index('key')
batsman_clus = batsman_clus.set_index('player_name')
bowler_clus = bowler_clus.set_index('player_name')
for i in range(1,6):
    clusters_pred.iloc[:,i]+=clusters_pred.iloc[:,i-1]

clusters_pred = clusters_pred[~clusters_pred.index.duplicated()]
batsman_clus = batsman_clus[~batsman_clus.index.duplicated()]
bowler_clus = bowler_clus[~bowler_clus.index.duplicated()]

In [5]:
def match(team_1n,team_2n,a,b,c,d):
#         team_1=pd.read_csv('team_csvs/'+team_1n+'.csv')
#         team_2=pd.read_csv('team_csvs/'+team_2n+'.csv')
#         team_1=team_1.drop('player_id',axis=1)
#         team_2=team_2.drop('player_id',axis=1)
#         team_1=team_1.dropna()
#         team_2=team_2.dropna()
#         bowlers_1=[]
#         for i in range(11):
#             if(team_1.iloc[i,20]=='Yes'):
#                 bowlers_1.append(team_1.iloc[i,0])
#         bowlers_2=[]
#         for i in range(11):
#             if(team_2.iloc[i,20]=='Yes'):
#                 bowlers_2.append(team_2.iloc[i,0])
#         batsman_1 = []
#         for i in range(11):
#             batsman_1.append(team_1.iloc[i,0])

#         batsman_2 = []
#         for i in range(11):
#             batsman_2.append(team_2.iloc[i,0])
        batsman_1 = a


        bowlers_1 = b


        batsman_2 = c

        bowlers_2 = d
        
        bowlers_1 = complete_bowling_order(bowlers_1)
        bowlers_2 = complete_bowling_order(bowlers_2)
        
        
        runs1=innings(batsman_1, bowlers_2,team_1n+'innings')
        runs2=innings(batsman_2, bowlers_1,team_2n+'innings')
        if runs1 == runs2:
            return 0
        elif runs1 > runs2:
            return 1
        else:
            return -1

In [6]:
# team_1n=input()
# team_2n=input()
# match(team_1n,team_2n)

In [7]:
#RCB V MI 2018
#Innings 1 RCB bat
Batsmen1 = ['M Vohra','Q de Kock','BB McCullum','V Kohli','AB de Villiers','C de Grandhomme','Washington Sundar','TG Southee','UT Yadav','YS Chahal','Mohammed Siraj']
# Bowlers = ['JP Duminy','MJ McClenaghan','JJ Bumrah','KH Pandya','HH Pandya','M Markande']
Bowlers2 = ['JP Duminy','MJ McClenaghan','JJ Bumrah','JP Duminy','MJ McClenaghan','KH Pandya','M Markande','KH Pandya','M Markande','HH Pandya','KH Pandya','M Markande','KH Pandya','JJ Bumrah','MJ McClenaghan','HH Pandya','JJ Bumrah','HH Pandya','JJ Bumrah','MJ McClenaghan']
#Innings Simulation
#RCB V MI 2018
#Innings 2 MI bat
Batsmen2 = ['SA Yadav','Ishan Kishan','JP Duminy','RG Sharma','KA Pollard','KH Pandya','HH Pandya','BCJ Cutting','MJ McClenaghan','M Markande','JJ Bumrah']
# Bowlers = ['TG Southee','UT Yadav','Mohammed Siraj','YS Chahal','Washington Sundar','C de Grandhomme']
Bowlers1 = ['TG Southee','UT Yadav','TG Southee','UT Yadav','Mohammed Siraj','UT Yadav','YS Chahal','Mohammed Siraj','YS Chahal','Washington Sundar','YS Chahal','C de Grandhomme','YS Chahal','C de Grandhomme','UT Yadav','C de Grandhomme','Mohammed Siraj','TG Southee','Mohammed Siraj','TG Southee']
result1 = 1
res1 = match('rcb','mi', Batsmen1, Bowlers1, Batsmen2, Bowlers2)

Over Number  1
The Bowler is MJ McClenaghan
M Vohra Has scored 0
M Vohra Has scored 1
Q de Kock Has scored 0
Q de Kock Has scored 0
Q de Kock Has scored 0
Q de Kock Has scored 0

OVER STATS
runs: 1 , outs 0

Over Number  2
The Bowler is JJ Bumrah
M Vohra Has scored 1
Q de Kock Has scored 0
Q de Kock Has scored 1
M Vohra Has scored 0
M Vohra Has scored 0
M Vohra Has scored 0

OVER STATS
runs: 2 , outs 0

Over Number  3
The Bowler is JP Duminy
Q de Kock Has scored 2
Q de Kock Has scored 0
Q de Kock got out
BB McCullum is the man next in
M Vohra Has scored 0
M Vohra Has scored 1
BB McCullum Has scored 0

OVER STATS
runs: 3 , outs 1

Over Number  4
The Bowler is MJ McClenaghan
M Vohra Has scored 6
M Vohra Has scored 1
BB McCullum Has scored 4
BB McCullum Has scored 0
BB McCullum Has scored 0
BB McCullum Has scored 0

OVER STATS
runs: 11 , outs 0

Over Number  5
The Bowler is KH Pandya
M Vohra got out
V Kohli is the man next in
BB McCullum Has scored 4
BB McCullum Has scored 0
BB McCullum H

HH Pandya Has scored 1
KH Pandya Has scored 2
KH Pandya Has scored 1
HH Pandya Has scored 0

OVER STATS
runs: 4 , outs 1

Over Number  17
The Bowler is TG Southee
KH Pandya Has scored 0
KH Pandya got out
BCJ Cutting is the man next in
BCJ Cutting Has scored 0
BCJ Cutting Has scored 4
BCJ Cutting Has scored 1
HH Pandya Has scored 0

OVER STATS
runs: 5 , outs 1

Over Number  18
The Bowler is Mohammed Siraj
BCJ Cutting Has scored 0
BCJ Cutting Has scored 0
BCJ Cutting Has scored 1
HH Pandya Has scored 1
BCJ Cutting Has scored 4
BCJ Cutting Has scored 4

OVER STATS
runs: 10 , outs 0

Over Number  19
The Bowler is TG Southee
HH Pandya Has scored 2
HH Pandya Has scored 1
BCJ Cutting Has scored 1
HH Pandya Has scored 4
HH Pandya Has scored 4
HH Pandya Has scored 0

OVER STATS
runs: 12 , outs 0

Over Number  20
The Bowler is TG Southee
BCJ Cutting Has scored 0
BCJ Cutting Has scored 4
BCJ Cutting Has scored 1
HH Pandya Has scored 0
HH Pandya Has scored 1
BCJ Cutting Has scored 1

OVER STATS
ru

In [8]:
# Three matches:

# First match - Sunrisers Hyderabad vs Mumbai Indians (SRH vs MI)

# 1st Innings
# Batting - SRH
# Bowling - MI

srh_batting = ['Wriddhiman Saha','Shikar Dhawan',
               'Kane Williamson','Manish Pandey',
               'Shakib Al Hasan','Deepak Hooda',
               'Yusuf Pathan','Rashid Khan',
               'Siddarth Kaul','Sandeep Sharma','Billy Stanlake']

mi_bowling = ['Pradeep Sangwan','Jasprit Bumrah',
              'Pradeep Sangwan','Ben Cutting',
              'Krunal Pandya','Jasprit Bumrah',
              'Mayank Markhande','Mustafizur Rahman',
              'Mayank Markhande','Krunal Pandya',
              'Mayank Markhande','Mustafizur Rahman',
              'Mayank Markhande','Ben Cutting',
              'Jasprit Bumrah','Mustafizur Rahman',
              'Ben Cutting','Jasprit Bumrah',
              'Mustafizur Rahman','Ben Cutting']

# 2nd Innings
# Batting - MI
# Bowling - SRH

mi_batting = ['Rohit Sharma','Evin Lewis',
              'Ishan Kishan','Suryakumar Yadav',
              'Krunal Pandya','Kieron Pollard',
              'Ben Cutting','Pradeep Sangwan',
              'Mayank Markhande','Jasprit Bumrah','Mustafizur Rahman']

srh_bowling = ['Sandeep Sharma','Billy Stanlake',
               'Sandeep Sharma','Billy Stanlake',
               'Rashid Khan','Siddarth Kaul',
               'Shakib Al Hasan','Sandeep Sharma',
               'Shakib Al Hasan','Billy Stanlake',
               'Rashid Khan','Siddarth Kaul',
               'Rashid Khan','Shakib Al Hasan',
               'Billy Stanlake','Shakib Al Hasan',
               'Siddarth Kaul','Rashid Khan',
               'Sandeep Sharma','Siddarth Kaul']
# team_1n=input()
# team_2n=input()
result2 = 1
res2 = match('srh','mi', srh_batting, srh_bowling, mi_batting, mi_bowling)

Over Number  1
The Bowler is Jasprit Bumrah
Wriddhiman Saha Has scored 0
Wriddhiman Saha Has scored 1
Shikar Dhawan Has scored 3
Wriddhiman Saha Has scored 4
Wriddhiman Saha Has scored 1
Shikar Dhawan Has scored 1

OVER STATS
runs: 10 , outs 0

Over Number  2
The Bowler is Pradeep Sangwan
Shikar Dhawan Has scored 2
Shikar Dhawan Has scored 4
Shikar Dhawan Has scored 1
Wriddhiman Saha Has scored 1
Shikar Dhawan Has scored 0
Shikar Dhawan Has scored 2

OVER STATS
runs: 10 , outs 0

Over Number  3
The Bowler is Ben Cutting
Wriddhiman Saha Has scored 0
Wriddhiman Saha Has scored 0
Wriddhiman Saha Has scored 4
Wriddhiman Saha Has scored 1
Shikar Dhawan Has scored 4
Shikar Dhawan Has scored 1

OVER STATS
runs: 10 , outs 0

Over Number  4
The Bowler is Krunal Pandya
Shikar Dhawan Has scored 1
Wriddhiman Saha Has scored 1
Shikar Dhawan Has scored 1
Wriddhiman Saha Has scored 1
Shikar Dhawan Has scored 1
Wriddhiman Saha Has scored 0

OVER STATS
runs: 5 , outs 0

Over Number  5
The Bowler is Jas

runs: 18 , outs 1

Over Number  14
The Bowler is Billy Stanlake
Krunal Pandya Has scored 0
Krunal Pandya Has scored 0
Krunal Pandya Has scored 1
Suryakumar Yadav got out
Kieron Pollard is the man next in
Kieron Pollard Has scored 4
Kieron Pollard Has scored 1

OVER STATS
runs: 6 , outs 1

Over Number  15
The Bowler is Shakib Al Hasan
Kieron Pollard Has scored 4
Kieron Pollard Has scored 0
Kieron Pollard Has scored 1
Krunal Pandya Has scored 0
Krunal Pandya Has scored 1
Kieron Pollard Has scored 0

OVER STATS
runs: 6 , outs 0

Over Number  16
The Bowler is Siddarth Kaul
Krunal Pandya Has scored 0
Krunal Pandya Has scored 1
Kieron Pollard Has scored 1
Krunal Pandya Has scored 1
Kieron Pollard Has scored 4
Kieron Pollard Has scored 1

OVER STATS
runs: 8 , outs 0

Over Number  17
The Bowler is Rashid Khan
Kieron Pollard Has scored 1
Krunal Pandya Has scored 2
Krunal Pandya Has scored 1
Kieron Pollard Has scored 1
Krunal Pandya Has scored 0
Krunal Pandya Has scored 6

OVER STATS
runs: 11 , 

In [9]:
# Second match - Kolkata Knight Riders vs Rajasthan Royals (KKR vs RR)

# 1st Innings
# Batting - RR
# Bowling - KKR

rr_batting = ['Rahul Tripathi','Jos Buttler',
              'Ajinkya Rahane','Sanju Samson',
              'Ben Stokes','Stuart Binny',
              'Krishanppa Gowtham','Jaydev Unadkat',
              'Ish Sodhi','Jofra Archer','Anureet Singh']

kkr_bowling = ['Shivam Mavi','Prasidh Krishna',
               'Shivam Mavi','Sunil Narine',
               'Andre Russell','Sunil Narine',
               'Andre Russell','Kuldeep Yadav',
               'Prasidh Krishna','Kuldeep Yadav',
               'Sunil Narine','Kuldeep Yadav',
               'Shivam Mavi','Kuldeep Yadav',
               'Shivam Mavi','Sunil Narine',
               'Prasidh Krishna','Andre Russell','Prasidh Krishna']

# 2nd Innings
# Batting - KKR
# Bowling - RR

kkr_batting = ['Sunil Narine','Chris Lynn',
               'Robin Uthappa','Nitish Rana',
               'Dinesh Karthik','Andre Russell',
               'Shubham Gill','Shivam Mavi',
               'Javon Searles','Prasidh Krishna','Kuldeep Yadav']

rr_bowling = ['Krishnappa Gowtham','Ben Stokes',
               'Jofra Archer','Ben Stokes',
               'Jofra Archer','Ben Stokes',
               'Ish Sodhi','Krishnappa Gowtham',
               'Ish Sodhi','Jaydev Unadkat',
               'Ish Sodhi','Jaydev Unadkat',
               'Anureet Singh','Jofra Archer',
               'Ish Sodhi','Ben Stokes',
               'Jaydev Unadkat','Jofra Archer']
# team_1n=input()
# team_2n=input()
result3 = -1
res3 = match('rr','kkr', rr_batting, rr_bowling, kkr_batting, kkr_bowling  )

Over Number  1
The Bowler is Prasidh Krishna
Rahul Tripathi Has scored 0
Rahul Tripathi Has scored 0
Rahul Tripathi Has scored 0
Rahul Tripathi Has scored 1
Jos Buttler Has scored 1
Rahul Tripathi Has scored 0

OVER STATS
runs: 2 , outs 0

Over Number  2
The Bowler is Shivam Mavi
Jos Buttler Has scored 1
Rahul Tripathi Has scored 0
Rahul Tripathi Has scored 0
Rahul Tripathi Has scored 4
Rahul Tripathi Has scored 1
Jos Buttler Has scored 0

OVER STATS
runs: 6 , outs 0

Over Number  3
The Bowler is Sunil Narine
Rahul Tripathi Has scored 1
Jos Buttler Has scored 6
Jos Buttler Has scored 0
Jos Buttler Has scored 0
Jos Buttler Has scored 0
Jos Buttler Has scored 1

OVER STATS
runs: 8 , outs 0

Over Number  4
The Bowler is Andre Russell
Jos Buttler Has scored 1
Rahul Tripathi Has scored 4
Rahul Tripathi Has scored 0
Rahul Tripathi Has scored 0
Rahul Tripathi Has scored 1
Jos Buttler got out
Ajinkya Rahane is the man next in

OVER STATS
runs: 6 , outs 1

Over Number  5
The Bowler is Sunil Nar

Dinesh Karthik Has scored 0
Dinesh Karthik Has scored 0
Dinesh Karthik Has scored 1
Andre Russell Has scored 1
Dinesh Karthik Has scored 0
Dinesh Karthik Has scored 1

OVER STATS
runs: 3 , outs 0

Over Number  15
The Bowler is Ben Stokes
Dinesh Karthik got out
Shubham Gill is the man next in
Shubham Gill Has scored 1
Andre Russell Has scored 0
Andre Russell Has scored 2
Andre Russell Has scored 0
Andre Russell Has scored 1

OVER STATS
runs: 4 , outs 1

Over Number  16
The Bowler is Jaydev Unadkat
Andre Russell Has scored 1
Shubham Gill Has scored 1
Andre Russell Has scored 1
Shubham Gill Has scored 1
Andre Russell Has scored 0
Andre Russell Has scored 4

OVER STATS
runs: 8 , outs 0

Over Number  17
The Bowler is Jofra Archer
Shubham Gill Has scored 4
Shubham Gill Has scored 1
Andre Russell Has scored 0
Andre Russell Has scored 1
Shubham Gill Has scored 6
Shubham Gill Has scored 0

OVER STATS
runs: 12 , outs 0

Over Number  18
The Bowler is Krishnappa Gowtham
Andre Russell Has scored 0


In [10]:
# Third match - Royal Challengers Bangalore vs Delhi Daredevils (RCB vs DD)

# 1st Innings
# Batting - DD
# Bowling - RCB

dd_batting = ['Jason Roy','Gautham Gambhir',
              'Shreyas Iyer','Rishab Pant',
              'Glenn Maxwell','Rahul Tewatia',
              'Chris Morris','Vijay Shankar',
              'Shahbaz Nadeem','Harshal Patel','Trent Boult']

rcb_bowling = ['Umesh Yadav','Chris Woakes',
               'Umesh Yadav','Yuzvendra Chahal',
               'Umesh Yadav','Yuzvendra Chahal',
               'Mohammed Siraj','Washington Sundar',
               'Umesh Yadav','Washington Sundar',
               'Chris Woakes','Washington Sundar',
               'Mohammed Siraj','Washington Sundar',
               'Mohammed Siraj','Yuzvendra Chahal',
               'Chris Woakes','Mohammed Siraj',
               'Chris Woakes','Corey Anderson']

# 2nd Innings
# Batting - RCB
# Bowling - DD

rcb_batting = ['Quinton De Kock','Manan Vohra',
               'Virat Kohli','AB de Villiers',
               'Corey Anderson','Mandeep Singh',
               'Washington Sundar','Chris Woakes',
               'Umesh Yadav','Mohammed Siraj','Yuzvendra Chahal']

dd_bowling = ['Trent Boult','Glenn Maxwell',
              'Trent Boult','Glenn Maxwell',
              'Shahbaz Nadeem','Chris Morris',
              'Shahbaz Nadeem','Chris Morris',
              'Harshal Patel','Rahul Tewatia',
              'Harshal Patel','Rahul Tewatia',
              'Harshal Patel','Rahul Tewatia',
              'Chris Morris','Trent Boult',
              'Chris Morris','Trent Boult']
# team_1n=input()
# team_2n=input()
result4 = -1
res4 = match('dd','rcb', dd_batting, dd_bowling, rcb_batting, rcb_bowling  )


Over Number  1
The Bowler is Chris Woakes
Jason Roy Has scored 4
Jason Roy Has scored 1
Gautham Gambhir Has scored 0
Gautham Gambhir Has scored 6
Gautham Gambhir Has scored 0
Gautham Gambhir Has scored 0

OVER STATS
runs: 11 , outs 0

Over Number  2
The Bowler is Umesh Yadav
Jason Roy Has scored 1
Gautham Gambhir Has scored 0
Gautham Gambhir Has scored 1
Jason Roy Has scored 1
Gautham Gambhir Has scored 4
Gautham Gambhir Has scored 1

OVER STATS
runs: 8 , outs 0

Over Number  3
The Bowler is Yuzvendra Chahal
Gautham Gambhir Has scored 2
Gautham Gambhir Has scored 2
Gautham Gambhir Has scored 0
Gautham Gambhir Has scored 1
Jason Roy Has scored 0
Jason Roy Has scored 1

OVER STATS
runs: 6 , outs 0

Over Number  4
The Bowler is Umesh Yadav
Jason Roy Has scored 1
Gautham Gambhir Has scored 4
Gautham Gambhir Has scored 1
Jason Roy Has scored 0
Jason Roy Has scored 1
Gautham Gambhir got out
Shreyas Iyer is the man next in

OVER STATS
runs: 7 , outs 1

Over Number  5
The Bowler is Yuzvendra C

Corey Anderson Has scored 1
Virat Kohli Has scored 0

OVER STATS
runs: 2 , outs 1

Over Number  14
The Bowler is Chris Morris
Corey Anderson Has scored 1
Virat Kohli Has scored 2
Virat Kohli Has scored 0
Virat Kohli Has scored 1
Corey Anderson Has scored 2
Corey Anderson Has scored 4

OVER STATS
runs: 10 , outs 0

Over Number  15
The Bowler is Trent Boult
Virat Kohli Has scored 0
Virat Kohli got out
Mandeep Singh is the man next in
Mandeep Singh Has scored 1
Corey Anderson Has scored 0
Corey Anderson Has scored 0
Corey Anderson Has scored 0

OVER STATS
runs: 1 , outs 1

Over Number  16
The Bowler is Chris Morris
Mandeep Singh Has scored 0
Mandeep Singh Has scored 4
Mandeep Singh Has scored 0
Mandeep Singh Has scored 1
Corey Anderson Has scored 1
Mandeep Singh Has scored 0

OVER STATS
runs: 6 , outs 0

Over Number  17
The Bowler is Trent Boult
Corey Anderson got out
Washington Sundar is the man next in
Washington Sundar Has scored 1
Mandeep Singh Has scored 0
Mandeep Singh Has scored 1


In [11]:
# Match 22

# Team1: 'KXIP'
# Team2: 'DD'

# Inn1:
kxipbat = ['KL Rahul','AJ Finch','MA Agarwal','KK Nair','Yuvraj Singh','DA Miller','R Ashwin','AJ Tye','BB Sran','AS Rajpoot','Mujeeb Ur Rahman']
ddbowl = ['TA Boult','Avesh Khan','TA Boult','Avesh Khan','LE Plunkett','Avesh Khan','DT Christian','LE Plunkett','DT Christian','A Mishra','GJ Maxwell','A Mishra','Avesh Khan','A Mishra','R Tewatia','A Mishra','LE Plunkett','DT Christian','LE Plunkett','TA Boult']
# Bowl = ['TA Boult','Avesh Khan','LE Plunkett','DT Christian','A Mishra','GJ Maxwell','R Tewatia']

# Inn2:
ddbat = ['PP Shaw','G Gambhir','GJ Maxwell','SS Iyer','RR Pant','DT Christian','R Tewatia','LE Plunkett', 'A Mishra','Avesh Khan','TA Boult']
# Bowl = ['AS Rajpoot','BB Sran','AJ Tye','R Ashwin','Mujeeb Ur Rahman']
kxipbowl = ['AS Rajpoot','BB Sran','AS Rajpoot','AJ Tye','AS Rajpoot','AJ Tye','AS Rajpoot','R Ashwin','Mujeeb Ur Rahman','R Ashwin','Mujeeb Ur Rahman','R Ashwin','BB Sran','R Ashwin','Mujeeb Ur Rahman','AJ Tye','BB Sran','AJ Tye','BB Sran','Mujeeb Ur Rahman']
print(len(kxipbat),len(ddbowl),len(ddbat), len(kxipbowl))
result5 = 1
res5 = match('kxip','dd', kxipbat, kxipbowl, ddbat, ddbowl  )

11 20 11 20
Over Number  1
The Bowler is Avesh Khan
KL Rahul Has scored 2
KL Rahul Has scored 6
KL Rahul Has scored 4
KL Rahul Has scored 0
KL Rahul Has scored 0
KL Rahul Has scored 0

OVER STATS
runs: 12 , outs 0

Over Number  2
The Bowler is TA Boult
AJ Finch Has scored 0
AJ Finch Has scored 4
AJ Finch Has scored 1
KL Rahul Has scored 1
AJ Finch Has scored 6
AJ Finch Has scored 1

OVER STATS
runs: 13 , outs 0

Over Number  3
The Bowler is Avesh Khan
AJ Finch Has scored 0
AJ Finch Has scored 1
KL Rahul Has scored 0
KL Rahul Has scored 0
KL Rahul Has scored 0
KL Rahul Has scored 1

OVER STATS
runs: 2 , outs 0

Over Number  4
The Bowler is LE Plunkett
KL Rahul Has scored 6
KL Rahul Has scored 0
KL Rahul got out
MA Agarwal is the man next in
AJ Finch Has scored 0
AJ Finch Has scored 4
AJ Finch Has scored 4

OVER STATS
runs: 14 , outs 1

Over Number  5
The Bowler is Avesh Khan
MA Agarwal Has scored 0
MA Agarwal Has scored 0
MA Agarwal Has scored 0
MA Agarwal Has scored 0
MA Agarwal Has sc

OVER STATS
runs: 7 , outs 0

Over Number  19
The Bowler is Mujeeb Ur Rahman
R Tewatia Has scored 1
DT Christian Has scored 0
DT Christian Has scored 1
R Tewatia Has scored 1
DT Christian Has scored 0
DT Christian Has scored 2

OVER STATS
runs: 5 , outs 0

Over Number  20
The Bowler is AS Rajpoot
R Tewatia Has scored 0
R Tewatia Has scored 4
R Tewatia Has scored 0
R Tewatia Has scored 1
DT Christian Has scored 1
R Tewatia Has scored 4

OVER STATS
runs: 10 , outs 0


BATSMAN STATS
{'PP Shaw': 7, 'G Gambhir': 22, 'GJ Maxwell': 21, 'SS Iyer': 12, 'RR Pant': 42, 'DT Christian': 11, 'R Tewatia': 20}
BOWLER STATS
{'AJ Tye': 2, 'Mujeeb Ur Rahman': 2, 'BB Sran': 1}

INNINGS OVER


In [12]:
#Match 25

# Team1: 'SRH'
# Team2: 'KXIP'

# Inn1:
Bat1 = ['S Dhawan','KS Williamson','WP Saha','MK Pandey','Shakib Al Hasan','YK Pathan','Mohammad Nabi','Rashid Khan','Basil Thampi','S Kaul','Sandeep Sharma']
# Bowl2 = ['AS Rajpoot','BB Sran','R Ashwin','MK Tiwary','Mujeeb Ur Rahman','AJ Tye']
Bowl2 = ['AS Rajpoot','BB Sran','AS Rajpoot','BB Sran','AS Rajpoot','BB Sran','R Ashwin','MK Tiwary','R Ashwin','Mujeeb Ur Rahman','AJ Tye','Mujeeb Ur Rahman','AJ Tye','Mujeeb Ur Rahman','R Ashwin','Mujeeb Ur Rahman','AJ Tye','R Ashwin','AJ Tye','AS Rajpoot']

# Inn2
Bat2 = ['KL Rahul','CH Gayle','MA Agarwal','KK Nair','AJ Finch','MK Tiwary','R Ashwin','AJ Tye','BB Sran','AS Rajpoot','Mujeeb Ur Rahman']
# Bowl1 = ['Sandeep Sharma','Mohammad Nabi','S Kaul','Basil Thampi','Rashid Khan','Shakib Al Hasan']
Bowl1 = ['Sandeep Sharma','Mohammad Nabi','Sandeep Sharma','Mohammad Nabi','Sandeep Sharma','S Kaul','Basil Thampi','Rashid Khan','Basil Thampi','Shakib Al Hasan','S Kaul','Rashid Khan','Shakib Al Hasan','Rashid Khan','Shakib Al Hasan','Sandeep Sharma','S Kaul','Rashid Khan','S Kaul','Basil Thampi']
result6 = 1
res6 = match('srh','kxip',Bat1,Bowl1,Bat2,Bowl2)

Over Number  1
The Bowler is BB Sran
S Dhawan Has scored 1
KS Williamson Has scored 4
KS Williamson Has scored 1
S Dhawan Has scored 6
S Dhawan Has scored 1
KS Williamson Has scored 6

OVER STATS
runs: 19 , outs 0

Over Number  2
The Bowler is AS Rajpoot
S Dhawan Has scored 2
S Dhawan Has scored 4
S Dhawan Has scored 4
S Dhawan Has scored 0
S Dhawan Has scored 4
S Dhawan Has scored 0

OVER STATS
runs: 14 , outs 0

Over Number  3
The Bowler is BB Sran
KS Williamson Has scored 4
KS Williamson Has scored 0
KS Williamson Has scored 4
KS Williamson Has scored 4
KS Williamson Has scored 0
KS Williamson Has scored 1

OVER STATS
runs: 13 , outs 0

Over Number  4
The Bowler is AS Rajpoot
KS Williamson Has scored 4
KS Williamson Has scored 0
KS Williamson Has scored 1
S Dhawan Has scored 2
S Dhawan Has scored 4
S Dhawan Has scored 4

OVER STATS
runs: 15 , outs 0

Over Number  5
The Bowler is BB Sran
KS Williamson got out
WP Saha is the man next in
WP Saha Has scored 6
WP Saha Has scored 0
WP Sah

OVER STATS
runs: 8 , outs 0

Over Number  18
The Bowler is S Kaul
AJ Tye Has scored 0
AJ Tye Has scored 1
R Ashwin Has scored 4
R Ashwin Has scored 1
AJ Tye Has scored 0
AJ Tye Has scored 4

OVER STATS
runs: 10 , outs 0

Over Number  19
The Bowler is Basil Thampi
R Ashwin Has scored 0
R Ashwin got out
BB Sran is the man next in
BB Sran Has scored 0
BB Sran Has scored 0
BB Sran Has scored 0
BB Sran Has scored 0

OVER STATS
runs: 0 , outs 1

Over Number  20
The Bowler is Sandeep Sharma
AJ Tye Has scored 0
AJ Tye Has scored 1
BB Sran Has scored 4
BB Sran Has scored 4
BB Sran Has scored 1
AJ Tye Has scored 6

OVER STATS
runs: 16 , outs 0


BATSMAN STATS
{'KL Rahul': 18, 'CH Gayle': 11, 'MA Agarwal': 7, 'KK Nair': 40, 'AJ Finch': 14, 'MK Tiwary': 7, 'R Ashwin': 15, 'AJ Tye': 16, 'BB Sran': 9}
BOWLER STATS
{'Sandeep Sharma': 1, 'Basil Thampi': 2, 'Shakib Al Hasan': 2, 'Rashid Khan': 1, 'S Kaul': 1}

INNINGS OVER


In [13]:
# Qualifier1:

# Team1 = 'SRH'
# Team2 = 'CSK'

# Inn2

Bat1= ['SR Watson','F du Plessis','SK Raina','AT Rayudu','MS Dhoni','DJ Bravo','RA Jadeja','DL Chahar','Harbhajan Singh','SN Thakur','L Ngidi']
# Bowl = ['B Kumar','Sandeep Sharma','S Kaul','CR Brathwaite','Rashid Khan','Shakib Al Hasan']
Bowl2 = ['B Kumar','Sandeep Sharma','B Kumar','S Kaul','B Kumar','S Kaul','CR Brathwaite','Rashid Khan','CR Brathwaite','Rashid Khan','Shakib Al Hasan','Rashid Khan','Sandeep Sharma','Shakib Al Hasan','Sandeep Sharma','Rashid Khan','S Kaul','CR Brathwaite','S Kaul','B Kumar']

# Inn1

Bat2 = ['S Dhawan','SP Goswami','KS Williamson','MK Pandey','Shakib Al Hasan','YK Pathan','CR Brathwaite','B Kumar','Rashid Khan','S Kaul','Sandeep Sharma']
# Bowl = ['DL Chahar','L Ngidi','SN Thakur','DJ Bravo','RA Jadeja']
Bowl1 = ['DL Chahar','L Ngidi','DL Chahar','L Ngidi','SN Thakur','DL Chahar','DJ Bravo','RA Jadeja','DJ Bravo','RA Jadeja','DL Chahar','RA Jadeja','SN Thakur','RA Jadeja','DJ Bravo','L Ngidi','DJ Bravo','SN Thakur','L Ngidi','SN Thakur']
result7 = -1
res7 = match('srh','csk',Bat1,Bowl1,Bat2,Bowl2)

Over Number  1
The Bowler is Sandeep Sharma
SR Watson Has scored 0
SR Watson Has scored 0
SR Watson Has scored 0
SR Watson Has scored 0
SR Watson Has scored 1
F du Plessis Has scored 0

OVER STATS
runs: 1 , outs 0

Over Number  2
The Bowler is B Kumar
SR Watson Has scored 1
F du Plessis Has scored 4
F du Plessis Has scored 0
F du Plessis Has scored 1
SR Watson Has scored 1
F du Plessis Has scored 4

OVER STATS
runs: 11 , outs 0

Over Number  3
The Bowler is S Kaul
SR Watson Has scored 0
SR Watson Has scored 4
SR Watson Has scored 1
F du Plessis Has scored 0
F du Plessis Has scored 0
F du Plessis Has scored 1

OVER STATS
runs: 6 , outs 0

Over Number  4
The Bowler is B Kumar
F du Plessis Has scored 6
F du Plessis Has scored 0
F du Plessis Has scored 1
SR Watson Has scored 1
F du Plessis Has scored 4
F du Plessis Has scored 0

OVER STATS
runs: 12 , outs 0

Over Number  5
The Bowler is S Kaul
SR Watson Has scored 0
SR Watson Has scored 1
F du Plessis Has scored 0
F du Plessis Has scored 0

YK Pathan Has scored 1
CR Brathwaite Has scored 1
YK Pathan Has scored 1
CR Brathwaite Has scored 4

OVER STATS
runs: 8 , outs 0

Over Number  18
The Bowler is L Ngidi
YK Pathan Has scored 1
CR Brathwaite got out
B Kumar is the man next in
YK Pathan Has scored 0
YK Pathan Has scored 4
YK Pathan Has scored 0
YK Pathan Has scored 0

OVER STATS
runs: 5 , outs 1

Over Number  19
The Bowler is SN Thakur
B Kumar Has scored 6
B Kumar Has scored 0
B Kumar Has scored 0
B Kumar Has scored 4
B Kumar Has scored 1
YK Pathan Has scored 1

OVER STATS
runs: 12 , outs 0

Over Number  20
The Bowler is DL Chahar
YK Pathan Has scored 1
B Kumar Has scored 1
YK Pathan Has scored 1
B Kumar Has scored 0
B Kumar Has scored 0
B Kumar Has scored 1

OVER STATS
runs: 4 , outs 0


BATSMAN STATS
{'S Dhawan': 25, 'SP Goswami': 21, 'KS Williamson': 13, 'MK Pandey': 3, 'Shakib Al Hasan': 27, 'YK Pathan': 28, 'CR Brathwaite': 22, 'B Kumar': 13}
BOWLER STATS
{'DL Chahar': 1, 'DJ Bravo': 2, 'SN Thakur': 1, 'RA Jadeja': 1,

In [14]:
print((result1==res1),(result2==res2),(result3==res3),(result4==res4),(result5==res5),(result6==res6),(result7==res7))

False True True False False True False


In [15]:
# Delhi daredevis vs Kolkata Knight Riders

# 1st innings 
# Batting - DD
# Bowling - KKR

dd_batting = ['PP Shaw','Colin Munro',
				'Shreyas Iyer','Rishabh Pant',
				'Glenn Maxwell','Vijay Shankar',
				'Rahul Tewatia','Liam Plunkett',
				'Amit Mishra','Avesh Khan','Trent Boult']

kkr_bowling = ['Piyush Chawla','Kuldeep Yadav',
			   'Piyush Chawla','Shivam Mavi',
			   'Sunil Narine','Mitchell Johnson',
			   'Shivam Mavi','Andre Russell',
			   'Mitchell Johnson','Sunil Narine',
			   'Andre Russell','Kuldeep Yadav',
			   'Mitchell Johnson','Piyush Chawla',
			   'Andre Russell','Piyush Chawla',
			   'Shivam Mavi','Sunil Narine',
			   'Mitchell Johnson','Shivam Mavi']

# 2st innings 
# Batting - KKR
# Bowling - DD

kkr_batting = ['Chris Lynn','Sunil Narine',
				'Robin Uthappa','Nitish Rana',
				'Dinesh Karthik','Shubman Gill',
				'Andre Russell','Shivam Mavi',
				'Piyush Chawla','Mitchell Johnson','Kuldeep Yadav']

dd_bowling = ['Trent Boult','Glenn Maxwell',
			  'Trent Boult','Avesh Khan',
			  'Liam Plunkett','Avesh Khan',
			  'Liam Plunkett','Amit Mishra',
			  'Vijay Shankar','Amit Mishra',
			  'Liam Plunkett','Avesh Khan',
			  'Trent Boult','Amit Mishra',
			  'Rahul Tewatia','Amit Mishra'
			  'Liam Plunkett','Avesh Khan'
			  'Glenn Maxwell','Trent Boult']
resultfin = 1 
resfin = match('dd','kkr',dd_batting,dd_bowling,kkr_batting,kkr_bowling)

Over Number  1
The Bowler is Kuldeep Yadav
PP Shaw Has scored 4
PP Shaw Has scored 0
PP Shaw Has scored 1
Colin Munro Has scored 0
Colin Munro Has scored 0
Colin Munro Has scored 1

OVER STATS
runs: 6 , outs 0

Over Number  2
The Bowler is Piyush Chawla
Colin Munro Has scored 4
Colin Munro Has scored 0
Colin Munro Has scored 2
Colin Munro Has scored 1
PP Shaw Has scored 1
Colin Munro Has scored 0

OVER STATS
runs: 8 , outs 0

Over Number  3
The Bowler is Shivam Mavi
PP Shaw Has scored 0
PP Shaw Has scored 1
Colin Munro Has scored 4
Colin Munro Has scored 1
PP Shaw Has scored 1
Colin Munro Has scored 0

OVER STATS
runs: 7 , outs 0

Over Number  4
The Bowler is Sunil Narine
PP Shaw Has scored 6
PP Shaw Has scored 1
Colin Munro Has scored 0
Colin Munro Has scored 1
PP Shaw Has scored 1
Colin Munro Has scored 1

OVER STATS
runs: 10 , outs 0

Over Number  5
The Bowler is Mitchell Johnson
Colin Munro Has scored 4
Colin Munro got out
Shreyas Iyer is the man next in
Shreyas Iyer Has scored 2
S

Shubman Gill Has scored 6
Shubman Gill Has scored 1
Nitish Rana Has scored 0
Nitish Rana Has scored 6
Nitish Rana Has scored 4
Nitish Rana Has scored 6

OVER STATS
runs: 23 , outs 0

Over Number  16
The Bowler is Avesh KhanGlenn Maxwell
Shubman Gill Has scored 1
Nitish Rana Has scored 1
Shubman Gill Has scored 0
Shubman Gill Has scored 1
Nitish Rana Has scored 1
Shubman Gill Has scored 1

OVER STATS
runs: 5 , outs 0

Over Number  17
The Bowler is Trent Boult
Shubman Gill Has scored 6
Shubman Gill Has scored 1
Nitish Rana Has scored 0
Nitish Rana Has scored 0
Nitish Rana got out
Andre Russell is the man next in
Andre Russell Has scored 1

OVER STATS
runs: 8 , outs 1

Over Number  18
The Bowler is Amit MishraLiam Plunkett
Andre Russell Has scored 4
Andre Russell Has scored 1
Shubman Gill Has scored 0
Shubman Gill Has scored 0
Shubman Gill Has scored 0
Shubman Gill Has scored 4

OVER STATS
runs: 9 , outs 0

Over Number  19
The Bowler is Glenn Maxwell
Andre Russell Has scored 0
Andre Russe

In [16]:
resfin == resultfin

True

In [19]:
pred_runs('PP Ojha','R Bhatia',random())

1